![image](https://github.com/ecastillot/SeisMonitor/blob/master/docs/figures/seismonitor.PNG?raw=true)

**This code is necessary on colab to install SeisMonitor.** 

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install SeisMonitor
    !pip install git+https://github.com/ecastillot/EQTransformer.git@master
    !pip install git+https://github.com/wayneweiqiang/GaMMA.git

**Please restart the kernel. It's mandatory to get everything up and running.**

Firstable, we need to use some files as input

- **velmodel**: CSV file with 1D Velocity model 
- **stations**: Metadata information. Feel free to use SeisMonitor.monitor.downloader.utils.get_inv_and_json to obtain inv.xml and stations.json information.
- **associations**: event.xml

Those files could be prepared in [Downloader](https://colab.research.google.com/github/ecastillot/SeisMonitor/blob/master/examples/1.downloader.ipynb), [Associator](https://colab.research.google.com/github/ecastillot/SeisMonitor/blob/master/examples/3.associator.ipynb)


To save time, we prepared one example to you. Feel free to see the format

In [ ]:
import os
from SeisMonitor.utils4examples import clone_seismonitor_data

monitor_path = os.path.join(os.path.dirname(os.getcwd()),"6m")
others = monitor_path = os.path.join(os.path.dirname(os.getcwd()),"others")

clone_seismonitor_data(others,branch="others")
velmodel = os.path.join(others,"velmodel")
stations = os.path.join(others,"stations")
asso = os.path.join(others,"events","eqt")


print("others dir: ",others)
print("Important folders in others directory",os.listdir(others))
print("#"*100)
print("velmodel dir: ",velmodel)
print("\tImportant files in velmodel directory",os.listdir(velmodel))
print("stations dir: ",stations)
print("\tImportant files in stations directory",os.listdir(stations))
print("asso dir: ",asso)
print("\tImportant files in asso directory",os.listdir(asso))

We will locate the seismic events in associations.xml using NonLinLoc

# Locate events with NonLinLoc

In [ ]:
import os
from SeisMonitor.monitor.locator.nlloc.nlloc import NLLoc
from SeisMonitor.monitor.locator import utils as lut

We create the next objects: VelModel and Stations to instance NLLoc class.

In [ ]:
out_dir = os.path.join(monitor_path,"nlloc")
vel_path = os.path.join(velmodel,"vel1d_col.csv")
inv = os.path.join(stations,"inv.xml")

vel_model = lut.VelModel(vel_path,model_name="Ojeda&Havskov(2004)")
stations = lut.Stations(inv)

As we are using Colab, we will install NLLoc. Just in case, this procedure has been tested just for linux:Ubuntu20 machines

In [ ]:
import platform
from SeisMonitor.monitor.locator.nlloc.utils import download_nlloc
if "linux" in platform.system().lower():
  test = ! lsb_release -a
  test = "__".join(test)
  if "ubuntu" in test.lower():
    ! sudo apt-get install -y cmake
    nlloc_path = "/opt/NLLoc"
    download_nlloc(nlloc_path)
    print("NLLoc now is working properly" )
  else:
    nlloc_path = None
    print("NLLoc was not installed using this script. Please install NonLinLoc from: https://github.com/alomax/NonLinLoc" )
else: 
    nlloc_path = None
    print("NLLoc was not installed using this script. Please install NonLinLoc from: https://github.com/alomax/NonLinLoc" )

Let's instantiate the NLLoc Class. 
- tmp folder is the path where we will place all travel times computed by NLLoc. So it will consume a lot of space in yor disk depending on the number of statiosn

In [ ]:
nlloc = NLLoc(
        core_path = nlloc_path, ### type your NLLoc path, 
        agency="SeisMonitor",
        region = [-85, -68,0, 15,-5, 205],
        vel_model = vel_model,
        stations = stations,
        delta_in_km = 2.5,
        tmp_folder=os.path.join(os.getcwd(),"NLLoc_grid") ### CHANGE PATH TO YOUR OWN PATH AND ALSO TAKE IN MIND THAT CONSUME DISK
        )

let's compute traveltimes for each station in our inventory. Take in mind that it consumes  disk memory depending on the number of stations. **It is going to take several minutes while Nlloc is computing the travel times**

In [ ]:
nlloc.compute_travel_times()
print(nlloc.tmp_folder)
print("nlloc.tmp_folder dir: ",nlloc.tmp_folder)
print("Important folders in nlloc.tmp_folder directory",os.listdir(nlloc.tmp_folder))

We are ready to locate the events

In [ ]:
eqt_catalog = os.path.join(asso,"associations.xml")
eqt_nlloc_catalog = nlloc.locate(catalog=eqt_catalog,
                            nlloc_out_folder= out_dir,
                            out_filename = "LOC.xml",
                            out_format="SC3ML" )
print(eqt_nlloc_catalog)